# Ensemble methods. Exercises


In this section we have only two exercise:

1. Find the best three classifier in the stacking method using the classifiers from scikit-learn package.

2. Build arcing arc-x4 method. 

In [1]:
%store -r data_set
%store -r labels
%store -r test_data_set
%store -r test_labels
%store -r unique_labels

## Exercise 1: Find the best three classifier in the stacking method

Please use the following classifiers:

* Linear regression,
* Nearest Neighbors,
* Linear SVM,
* Decision Tree,
* Naive Bayes,
* QDA.

In [2]:
import numpy as np
from sklearn.metrics import accuracy_score

from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

In [3]:
def build_classifiers():
    classifiers = []
    
    classifiers.append(LinearRegression())
    classifiers.append(KNeighborsClassifier())
    classifiers.append(SVC())
    classifiers.append(DecisionTreeClassifier())
    classifiers.append(GaussianNB())
    classifiers.append(QuadraticDiscriminantAnalysis())
    
    return classifiers

In [4]:
def build_stacked_classifier(classifiers):
    output = []
    for classifier in classifiers:
        classifier.fit(data_set, labels)
        output.append(classifier.predict(data_set))
    output = np.array(output).reshape((len(data_set), len(classifiers)))
    
    stacked_classifier = DecisionTreeClassifier()
    stacked_classifier.fit(output, labels)
    
    test_set = []
    for classifier in classifiers:
        test_set.append(classifier.predict(test_data_set))
    test_set = np.array(test_set).reshape((len(test_data_set), len(classifiers)))
    
    predicted = stacked_classifier.predict(test_set)
    return predicted

In [5]:
classifiers = build_classifiers()
predicted = build_stacked_classifier(classifiers)
accuracy = accuracy_score(test_labels, predicted)
print(accuracy)

0.6


In [6]:
from itertools import permutations
from copy import deepcopy
import pandas as pd

classifiers = build_classifiers()
accuracies = []
seen_permutations = set()

for selected_classifiers in permutations(classifiers, 3):
    three_classifiers = selected_classifiers[:3]

    unique_permutation = tuple(sorted([str(classifier) for classifier in three_classifiers]))
    if unique_permutation not in seen_permutations:
        seen_permutations.add(unique_permutation)
        predicted = build_stacked_classifier(three_classifiers)
        accuracy = accuracy_score(test_labels, predicted)
        accuracies.append((accuracy, str(type(three_classifiers[0]).__name__), str(type(three_classifiers[1]).__name__), str(type(three_classifiers[2]).__name__)))

sorted_accuracies = sorted(accuracies, key=lambda x: x[0], reverse=True)
df = pd.DataFrame(sorted_accuracies,
                  columns=['Accuracy', 'Classifier 1', 'Classifier 2', 'Classifier 3'])

df.head(10)

,Accuracy,Classifier 1,Classifier 2,Classifier 3
0,0.95,KNeighborsClassifier,DecisionTreeClassifier,QuadraticDiscriminantAnalysis
1,0.85,KNeighborsClassifier,SVC,GaussianNB
2,0.85,KNeighborsClassifier,SVC,QuadraticDiscriminantAnalysis
3,0.85,KNeighborsClassifier,GaussianNB,QuadraticDiscriminantAnalysis
4,0.80,KNeighborsClassifier,SVC,DecisionTreeClassifier
5,0.80,DecisionTreeClassifier,GaussianNB,QuadraticDiscriminantAnalysis
6,0.75,LinearRegression,KNeighborsClassifier,QuadraticDiscriminantAnalysis
7,0.75,LinearRegression,DecisionTreeClassifier,QuadraticDiscriminantAnalysis
8,0.75,SVC,DecisionTreeClassifier,QuadraticDiscriminantAnalysis
9,0.75,SVC,GaussianNB,QuadraticDiscriminantAnalysis


## Exercise 2: 

Use the boosting method and change the code to fullfilt the following requirements:

* the weights should be calculated as:
$w_{n}^{(t+1)}=\frac{1+ I(y_{n}\neq h_{t}(x_{n})}{\sum_{i=1}^{N}1+I(y_{n}\neq h_{t}(x_{n})}$,
* the prediction is done with a voting method.

In [7]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier

# prepare data set

def generate_data(sample_number, feature_number, label_number):
    data_set = np.random.random_sample((sample_number, feature_number))
    labels = np.random.choice(label_number, sample_number)
    return data_set, labels

labels = 2
dimension = 2
test_set_size = 1000
train_set_size = 5000
train_set, train_labels = generate_data(train_set_size, dimension, labels)
test_set, test_labels = generate_data(test_set_size, dimension, labels)

# init weights
number_of_iterations = 10
weights = np.ones((test_set_size,)) / test_set_size


def train_model(classifier, weights):
    return classifier.fit(X=test_set, y=test_labels, sample_weight=weights)

def calculate_error(model):
    predicted = model.predict(test_set)
    I=calculate_accuracy_vector(predicted, test_labels)
    Z=np.sum(I)
    return (1+Z)/1.0

def calculate_accuracy_vector(predicted, labels):
    result = []
    for i in range(len(predicted)):
        if predicted[i] == labels[i]:
            result.append(1)
        else:
            result.append(0)
    return result

Fill the two functions below:

In [8]:
def set_new_weights(model):
    I = np.array(calculate_accuracy_vector(model.predict(test_set), test_labels))
    return (1 + I) / (1 + I).sum()

Train the classifier with the code below:

In [9]:
classifier = DecisionTreeClassifier(max_depth=1, random_state=1)
classifier.fit(X=train_set, y=train_labels)
alphas = []
classifiers = []
for iteration in range(number_of_iterations):
    model = train_model(classifier, weights)
    weights = set_new_weights(model)
    classifiers.append(model)

print(weights)


validate_x, validate_label = generate_data(1, dimension, labels)

[0.00065359 0.00065359 0.00130719 0.00130719 0.00065359 0.00130719
 0.00065359 0.00065359 0.00130719 0.00130719 0.00130719 0.00130719
 0.00130719 0.00130719 0.00065359 0.00130719 0.00130719 0.00130719
 0.00065359 0.00130719 0.00065359 0.00065359 0.00130719 0.00130719
 0.00065359 0.00065359 0.00130719 0.00130719 0.00065359 0.00130719
 0.00130719 0.00065359 0.00065359 0.00130719 0.00065359 0.00065359
 0.00130719 0.00065359 0.00065359 0.00130719 0.00065359 0.00065359
 0.00065359 0.00065359 0.00065359 0.00130719 0.00130719 0.00065359
 0.00130719 0.00130719 0.00065359 0.00130719 0.00065359 0.00130719
 0.00065359 0.00065359 0.00130719 0.00065359 0.00130719 0.00065359
 0.00065359 0.00065359 0.00065359 0.00065359 0.00065359 0.00065359
 0.00065359 0.00065359 0.00065359 0.00065359 0.00130719 0.00130719
 0.00065359 0.00130719 0.00065359 0.00130719 0.00065359 0.00065359
 0.00130719 0.00130719 0.00065359 0.00130719 0.00130719 0.00130719
 0.00065359 0.00130719 0.00130719 0.00065359 0.00065359 0.0013

Set the validation data set:

In [10]:
validate_x, validate_label = generate_data(1, dimension, labels)

Fill the prediction code:

In [11]:
def get_prediction(x):
    output = np.array([classifier.predict(x) for classifier in classifiers])
    predicted = []
    for i in range(len(x)):
        classified = output[:, i]
        predicted.append(np.argmax(np.bincount(classified)))
    return predicted

Test it:

In [12]:
prediction = get_prediction(validate_x)[0]

print(prediction)

1
